In [1]:
import numpy as np
import pandas as pd
from keras.utils import to_categorical
from keras_preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os

In [2]:
TRAIN_DIR = 'images/train'
TEST_DIR = 'images/test'

In [3]:
def label_images(dir):
    image_path = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_path.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label,'completed')
    return image_path, labels

In [4]:
train = pd.DataFrame()
train['image'], train['label'] = label_images(TRAIN_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [5]:
test = pd.DataFrame()
test['image'], test['label'] = label_images(TEST_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [6]:
from tqdm.notebook import tqdm

In [7]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image, grayscale=True)
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features),48,48,1)
    return features

In [8]:
train_features = extract_features(train['image'])
test_features = extract_features(test['image'])

  0%|          | 0/28821 [00:00<?, ?it/s]

F:\anaconda\envs\face_detection_env\lib\site-packages\keras_preprocessing\image\utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


  0%|          | 0/7066 [00:00<?, ?it/s]

In [9]:
x_train = train_features/255.0
x_test = test_features/255.0

In [10]:
from sklearn.preprocessing import LabelEncoder

In [11]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [12]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [13]:
y_train = to_categorical(y_train, num_classes=7)
y_test = to_categorical(y_test, num_classes=7)

In [14]:
model = Sequential()
#convolution layer
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
#fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
#output class
model.add(Dense(7, activation='softmax'))

In [15]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
model.fit(x=x_train, y=y_train, batch_size=32, epochs=100, validation_data=(x_test,y_test))

Epoch 1/100
901/901 [==============================] - 968s 1s/step - loss: 1.7952 - accuracy: 0.2552 - val_loss: 1.6770 - val_accuracy: 0.3132
Epoch 2/100
901/901 [==============================] - 965s 1s/step - loss: 1.6288 - accuracy: 0.3570 - val_loss: 1.5279 - val_accuracy: 0.4304
Epoch 3/100
901/901 [==============================] - 957s 1s/step - loss: 1.5198 - accuracy: 0.4061 - val_loss: 1.4057 - val_accuracy: 0.4645
Epoch 4/100
901/901 [==============================] - 952s 1s/step - loss: 1.4551 - accuracy: 0.4339 - val_loss: 1.3465 - val_accuracy: 0.4884
Epoch 5/100
901/901 [==============================] - 975s 1s/step - loss: 1.4085 - accuracy: 0.4567 - val_loss: 1.2944 - val_accuracy: 0.5007
Epoch 6/100
901/901 [==============================] - 933s 1s/step - loss: 1.3796 - accuracy: 0.4681 - val_loss: 1.2830 - val_accuracy: 0.5146
Epoch 7/100
901/901 [==============================] - 523s 581ms/step - loss: 1.3402 - accuracy: 0.4869 - val_loss: 1.2437 - val_accura

In [18]:
model_json = model.to_json()
with open("emotiondetector.json","w") as json_file:
    json_file.write(model_json)
model.save("emotiondetector.h5")

In [21]:
from tensorflow.keras.models import model_from_json

In [22]:
json_file = open("emotiondetector.json","r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("emotiondetector.h5")

In [23]:
label = ['angry','disgust','fear','happy','neutral','sad','surprise']

In [26]:
def ef(image):
    img = load_img(image, grayscale = True)
    feature = np.array(img)
    feature = feature.reshape(1,48,48,1)
    return feature/255.0

In [31]:
image = "images/train/fear/2.jpg"
print("original image is of fear")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)

original image is of fear
1/1 [==============================] - 0s 31ms/step
model prediction is  fear
